# Portfolio optimization
## 基于动态风险控制的组合优化模型
在平衡收益，风险的基础上，动态调整风格、行业约束以及跟踪误差阈值，通过解决凸优化问题来确定组合内的股票权重，使得组合取得更高、更稳健的收益。
本程序主要是建立在Stanford开源库cvxopt的基础上，通过确定目标函数，添加风格和行业约束来求解股票权重。

maximize $$ \mu^Tw - γ(f^T \Sigma f+w^TDw) $$

subject to $$1^Tw=1, f=F^Tw, w∈W, f∈F, g∈G$$

where the variables are the allocations $w∈R^n$ and factor exposures $f∈R^k$ and $F$ gives the factor exposure constraints, $G$ gives the industry constraints, $γ$ is the risk aversion parameter, $\Sigma$ is the factor return covariance matrix, $D$ is the specific risk return.

### 1. load modules

In [113]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import re
from lib.gftTools import gftIO
from cvxopt import matrix, solvers, spmatrix, sparse
from cvxopt.blas import dot
import os
import uuid
from lib.gftTools import gsConst
solvers.options['show_progress'] = False

In [2]:
import logging
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

### 2. load data

In [86]:
target_date = gftIO.zload(os.path.join(path, 'target_date.pkl'))
target_date = pd.to_datetime(target_date)
target_date

Timestamp('2016-10-31 00:00:00')

In [164]:
target_return = 0.02
target_risk = 0.05
position_limit = 100 # only take 100 out of 3384 stocks.

In [13]:
asset_return = gftIO.zload(os.path.join(path, 'asset_return.pkl'))
asset_return = asset_return.asMatrix()

In [15]:
asset_weight = gftIO.zload(os.path.join(path, 'asset_weight.pkl'))
asset_weights = asset_weight.asColumnTab()
asset_weights

,date,symbol,value,industry
0,2015-07-31,b'\xecC\x1e\xb1\xa5\x11\x01\x01x\xa6\xc0&z\xec...,0.001916,b'\xa8B7\xd6\x1aW\xe9W\x02\x00\x00\x7f\x00\x00...
1,2016-07-29,"b'8H\x8c\x1e\xd9\x834\x02~,\x82\xce\xfeu(\x98'",0.001848,b'\xa8B7\xd6\xfcU\xe9W\x02\x00\x00\x7f\x00\x00...
2,2016-08-31,"b'8H\x8c\x1e\xd9\x834\x02~,\x82\xce\xfeu(\x98'",0.001845,b'\xa8B7\xd6\xfcU\xe9W\x02\x00\x00\x7f\x00\x00...
3,2016-09-30,"b'8H\x8c\x1e\xd9\x834\x02~,\x82\xce\xfeu(\x98'",0.001832,b'\xa8B7\xd6\xfcU\xe9W\x02\x00\x00\x7f\x00\x00...
4,2016-10-31,"b'8H\x8c\x1e\xd9\x834\x02~,\x82\xce\xfeu(\x98'",0.001828,b'\xa8B7\xd6\xfcU\xe9W\x02\x00\x00\x7f\x00\x00...
5,2016-11-30,"b'8H\x8c\x1e\xd9\x834\x02~,\x82\xce\xfeu(\x98'",0.001802,b'\xa8B7\xd6\xfcU\xe9W\x02\x00\x00\x7f\x00\x00...
6,2016-07-29,b'nKv\xc5%\xb5\x9f\x02\xadY/9lE\xcd\x82',0.001848,b'\xa8B7\xd6yV\xe9W\x02\x00\x00\x7f\x00\x00\x0...
7,2015-08-31,"b'\x1dF""\xda&f\xee\x03RK\xf5\xe1i\xc2\x8f\x88'",0.001942,b'\xa8B7\xd6\xf8U\xe9W\x02\x00\x00\x7f\x00\x00...
8,2015-09-30,"b'\x1dF""\xda&f\xee\x03RK\xf5\xe1i\xc2\x8f\x88'",0.001942,b'\xa8B7\xd6\xf8U\xe9W\x02\x00\x00\x7f\x00\x00...
9,2015-10-30,"b'\x1dF""\xda&f\xee\x03RK\xf5\xe1i\xc2\x8f\x88'",0.001965,b'\xa8B7\xd6\xf8U\xe9W\x02\x00\x00\x7f\x00\x00...


In [14]:
asset_return

,b'\x8eK\x9c+\x8a\xc71\x00j\xfb\x07AX\x13\xc8\xac',b'\xecC\x1e\xb1\xa5\x11\x01\x01x\xa6\xc0&z\xec\xba\xb0',"b'8H\x8c\x1e\xd9\x834\x02~,\x82\xce\xfeu(\x98'",b'\xb7Hy<\xa9\xd8~\x02\x10\xcd\xa8\x1d\xc1\xed4\xa9',b'nKv\xc5%\xb5\x9f\x02\xadY/9lE\xcd\x82',b'\xb1A\x99\xa3fy\x07\x03\xe1\x0f\xdb\xf3=\xf6\xbf\xaa',"b'\x1dF""\xda&f\xee\x03RK\xf5\xe1i\xc2\x8f\x88'","b""\x08M\x0e'\x1e\xe8\xf8\x04\x89pD\x87\x80\xcf\x8d\xa1""",b'#K\xa26\xd5\x16\x1a\x057T1\xba\xb7w \xb4',b'\xe9C\x11\xfa0\x98h\x05\xc4~\xd9\x14\xa1\xce\xc5\x91',...,b'\x0eA\x0eU#g\x10\xf8\xe5\x1a\xc8i\x8b\xee:\xbd',"b'\xa0@K\xbc\xa6F,\xf8^O\xcd\xect\xfbU\x95'",b'\x1cK\xca;\xf3(\x9a\xf8m\xaf\t~R\xb6\x1e\xbf',b'@A\x953\xdf\x8e\xe3\xf8\x82\xc8\xfd\xc9:\xf0\xcb\x8c',b'NA\x03\xb0dI1\xf9\xf0\xea\xf1y\xe7\x10\xe1\xb7',"b'\xa2H\xbd\x8cZZY\xf9\xce#\xf6$""!\x05\x99'",b'bM\xcd\t|\xb5\x85\xf9u\xddi=\x89\xffO\x8d',b'\xc9M\xcaX\x11\xe2\x92\xfb\x86\xc2\rf\xcb\xe79\x9f',b'AA\xd6\xbe1\xe8:\xfd\x83D\xaa\xf3\\\r\xb0\xb6',b'\xafG\xc4\xf9\xe0\xeay\xfexD\x7f*\x85Ca\xa1'
2015-06-03,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,-0.038921,NaN,NaN,...,-0.024538,NaN,0.025354,-0.031156,NaN,0.050749,NaN,NaN,0.100086,NaN
2015-06-04,NaN,NaN,NaN,NaN,NaN,NaN,0.100059,-0.042205,NaN,NaN,...,-0.084752,NaN,-0.083799,0.003112,NaN,-0.076428,NaN,NaN,0.100047,NaN
2015-06-05,NaN,NaN,NaN,NaN,NaN,NaN,0.100080,0.007896,NaN,NaN,...,0.002326,NaN,0.010096,-0.009824,NaN,0.091772,NaN,NaN,0.100043,NaN
2015-06-08,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.006571,NaN,NaN,...,-0.072769,NaN,-0.071153,NaN,NaN,-0.084180,NaN,NaN,0.099987,NaN
2015-06-09,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.009038,NaN,NaN,...,0.023885,NaN,0.019817,NaN,NaN,-0.052853,NaN,NaN,-0.098415,NaN
2015-06-10,NaN,NaN,NaN,NaN,NaN,NaN,0.099980,0.024882,NaN,0.440151,...,0.033992,NaN,0.046908,NaN,NaN,0.031708,NaN,NaN,0.100039,NaN
2015-06-11,NaN,NaN,NaN,NaN,NaN,NaN,0.100055,0.018208,NaN,0.100131,...,0.012033,NaN,0.026145,NaN,NaN,0.008352,NaN,NaN,0.100059,NaN
2015-06-12,NaN,NaN,NaN,NaN,NaN,NaN,0.018657,-0.026466,NaN,0.099941,...,-0.005520,NaN,-0.068560,NaN,NaN,-0.002975,NaN,NaN,-0.063509,NaN
2015-06-15,NaN,NaN,NaN,NaN,NaN,NaN,-0.099918,-0.058290,NaN,0.100054,...,-0.081981,NaN,-0.100021,-0.008877,NaN,-0.032102,NaN,NaN,-0.100000,NaN
2015-06-16,NaN,NaN,NaN,NaN,NaN,NaN,-0.077580,-0.064239,NaN,0.099803,...,-0.076977,NaN,-0.018561,-0.033720,NaN,-0.035000,NaN,NaN,-0.042146,NaN


In [91]:
asset_names = asset_return.columns.unique()
asset_names

Index([      b'\x8eK\x9c+\x8a\xc71\x00j\xfb\x07AX\x13\xc8\xac',
       b'\xecC\x1e\xb1\xa5\x11\x01\x01x\xa6\xc0&z\xec\xba\xb0',
                b'8H\x8c\x1e\xd9\x834\x02~,\x82\xce\xfeu(\x98',
          b'\xb7Hy<\xa9\xd8~\x02\x10\xcd\xa8\x1d\xc1\xed4\xa9',
                      b'nKv\xc5%\xb5\x9f\x02\xadY/9lE\xcd\x82',
       b'\xb1A\x99\xa3fy\x07\x03\xe1\x0f\xdb\xf3=\xf6\xbf\xaa',
                b'\x1dF"\xda&f\xee\x03RK\xf5\xe1i\xc2\x8f\x88',
       b"\x08M\x0e'\x1e\xe8\xf8\x04\x89pD\x87\x80\xcf\x8d\xa1",
                   b'#K\xa26\xd5\x16\x1a\x057T1\xba\xb7w \xb4',
       b'\xe9C\x11\xfa0\x98h\x05\xc4~\xd9\x14\xa1\xce\xc5\x91',
       ...
             b'\x0eA\x0eU#g\x10\xf8\xe5\x1a\xc8i\x8b\xee:\xbd',
                   b'\xa0@K\xbc\xa6F,\xf8^O\xcd\xect\xfbU\x95',
               b'\x1cK\xca;\xf3(\x9a\xf8m\xaf\t~R\xb6\x1e\xbf',
       b'@A\x953\xdf\x8e\xe3\xf8\x82\xc8\xfd\xc9:\xf0\xcb\x8c',
             b'NA\x03\xb0dI1\xf9\xf0\xea\xf1y\xe7\x10\xe1\xb7',
                   b'\xa2H\xb

In [155]:
exposure_constraint = gftIO.zload(os.path.join(path, 'exposure_constraint.pkl'))
group_constraint = gftIO.zload(os.path.join(path, 'group_constraint.pkl'))
exposure_constraint = exposure_constraint.asMatrix()
asset_constraint = gftIO.zload(os.path.join(path, 'asset_constraint.pkl'))

#### Generate data for factor model.

#### In the following code we generate and solve a portfolio optimization problem with 30 factors and 100 out of 3384 assets.

In [116]:
factor_names = [uuid.uuid4().hex.upper()[0:6] for i in range(m)]

In [121]:
n = asset_names.shape[0]
m = 30 # factor number
np.random.seed(1)
mu = np.random.randn(n, 1)/100
D = np.diag(np.random.uniform(0, 0.9, size=asset_names.shape))/10 # specific risk return
D = pd.DataFrame(D, index=asset_names, columns=asset_names)
F = np.random.randn(n, m)/100 
F = pd.DataFrame(F, index=asset_names, columns=factor_names) # factor return
Sigma_tilde = np.matrix(np.cov(F.T)) # factor return covariance
Sigma_tilde = Sigma_tilde.T.dot(Sigma_tilde)


In [122]:
F

,F46E08,487A9C,06DB35,518D4A,1BCD6C,CA950D,6B9172,CDC4B7,44BA3A,D899C4,...,B1A177,32BAF1,F332E0,56B894,7D4AE9,46A9EF,D6F0D0,54F929,7C864D,BF8D42
b'\x8eK\x9c+\x8a\xc71\x00j\xfb\x07AX\x13\xc8\xac',0.001843,-0.001934,0.014174,-0.003698,-0.014925,0.005482,0.005441,-0.007461,-0.002392,-0.003285,...,-0.013001,-0.010779,0.002781,0.000464,0.008377,-0.004867,-0.012207,0.003189,0.000650,-0.006449
b'\xecC\x1e\xb1\xa5\x11\x01\x01x\xa6\xc0&z\xec\xba\xb0',0.004172,-0.005392,-0.014617,0.006170,-0.012204,0.004527,-0.006974,0.007237,0.009274,-0.011304,...,-0.016395,0.002452,0.010590,-0.011445,-0.001472,-0.019518,0.002127,0.001082,0.001313,-0.013638
"b'8H\x8c\x1e\xd9\x834\x02~,\x82\xce\xfeu(\x98'",0.002392,0.016767,0.006955,0.008092,0.007378,0.007299,-0.001195,0.010060,-0.011604,-0.001323,...,0.016024,0.000397,-0.000128,-0.018720,-0.004609,-0.018346,0.003963,-0.015459,0.015034,-0.003438
b'\xb7Hy<\xa9\xd8~\x02\x10\xcd\xa8\x1d\xc1\xed4\xa9',-0.016184,-0.009802,-0.015319,0.004666,0.001722,0.001402,-0.004540,0.010206,0.003128,-0.010563,...,-0.004737,0.002559,-0.006614,-0.004892,-0.010460,0.005973,0.006259,-0.000357,0.006317,0.010934
b'nKv\xc5%\xb5\x9f\x02\xadY/9lE\xcd\x82',-0.004417,0.007463,-0.003605,0.005337,0.009136,0.002047,-0.000365,-0.009019,-0.017962,0.013318,...,0.002715,-0.012855,0.012835,0.005516,0.012270,0.020662,-0.010200,0.005499,-0.004262,-0.005625
b'\xb1A\x99\xa3fy\x07\x03\xe1\x0f\xdb\xf3=\xf6\xbf\xaa',0.002830,0.008627,-0.012951,0.010514,0.006048,-0.015011,-0.000517,-0.013099,-0.009248,-0.010167,...,0.013616,-0.006471,-0.001549,0.004594,-0.000424,0.005422,0.003595,-0.008067,0.011793,0.001176
"b'\x1dF""\xda&f\xee\x03RK\xf5\xe1i\xc2\x8f\x88'",-0.002218,-0.001465,0.009389,0.005747,0.012755,0.006150,0.008339,0.002798,-0.010230,0.001330,...,0.000253,-0.002747,0.017694,-0.005241,0.007604,0.002008,-0.010383,0.006310,-0.001609,-0.007958
"b""\x08M\x0e'\x1e\xe8\xf8\x04\x89pD\x87\x80\xcf\x8d\xa1""",0.002866,0.004243,0.003902,0.020044,0.007353,-0.002814,-0.014044,-0.005296,0.009464,0.011172,...,0.020302,-0.002400,0.009502,0.005447,0.014287,-0.011193,-0.021563,0.012229,0.016182,-0.014628
b'#K\xa26\xd5\x16\x1a\x057T1\xba\xb7w \xb4',-0.006201,-0.004944,-0.010225,-0.024797,-0.009360,-0.011120,0.002951,0.021343,0.017962,-0.001735,...,-0.004944,-0.007527,-0.000127,-0.004697,-0.019319,0.016936,-0.006783,0.008730,-0.006436,-0.015551
b'\xe9C\x11\xfa0\x98h\x05\xc4~\xd9\x14\xa1\xce\xc5\x91',0.014689,-0.006157,0.004223,0.008608,0.006270,0.005618,-0.012933,0.020549,0.004847,-0.005705,...,-0.007269,0.008055,0.009408,-0.002363,-0.006270,0.002352,-0.001030,-0.009555,-0.001016,-0.002147


### 3. transform data, print statistics of return, normality test

In [123]:
# drop duplicated rows at date
df_industries_asset_weight = asset_weights.drop_duplicates(
    subset=['date', 'symbol'])
try:
    df_industries_asset_init_weight = df_industries_asset_weight[
        df_industries_asset_weight['date'] == target_date].dropna()
except KeyError:
    raise KeyError('invalid input date: %s' % target_date)

In [127]:
# drop incomplete rows
df_industries_asset_init_weight = df_industries_asset_init_weight.dropna(
    axis=0, subset=['industry', 'symbol'], how='any')

unique_symbol = df_industries_asset_init_weight['symbol'].unique()

In [130]:
# get random symbols at the target position limit
arr = list(range(len(unique_symbol )))
np.random.shuffle(arr)
target_symbols = unique_symbol[arr[:position_limit]]
target_symbols 

array([b'\xe2\x11\xeaH0N0\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11\xeaHaa/\xc1"\'i\xcb+xu\x90',
       b'\xc6Dl#q\x18\xeb\x1e\xb9\xe9L/\n7\x8c\x99',
       b'\xe2\x11\xeaH"!.\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11\xeaH\xc3\xfb=\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11\xeaHs\xff>\xc1"\'i\xcb+xu\x90',
       b'\x99A \xbbBd\x05\xa8\x16O\xdb\x11}\xd0=\xb4',
       b'\xe2\x11\xeaH\xe5<\x0b\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11\xeaH\xae\xd78\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11\xeaH\xa8YC\xc1"\'i\xcb+xu\x90',
       b'\xcfK\x9eg\xd0P\x87E\xd4\xa5-\xfd(PI\xbb',
       b'\xe2\x11\xeaH\x03\xea7\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11\xeaHwV\x17\xc1"\'i\xcb+xu\x90',
       b'\x0cG\x80g\xa6\x88\x1f\xa40\x97\x05\xb9\xc7?p\x85',
       b'\xe2\x11\xeaH\x1e\xde.\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11\xeaH\xe2\xcd>\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11\xeaH\x0c\x90C\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11\xeaH\x05\x05E\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11\xeaH{8&\xc1"\'i\xcb+xu\x90',
       b'\xe2\x11

In [131]:
df_industries_asset_target_init_weight = df_industries_asset_init_weight.\
                                         loc[df_industries_asset_init_weight['symbol'].isin(target_symbols)]
df_pivot_industries_asset_weights = pd.pivot_table(
    df_industries_asset_target_init_weight, values='value', index=['date'],
    columns=['industry', 'symbol'])
df_pivot_industries_asset_weights = df_pivot_industries_asset_weights.fillna(0)
logger.debug("set OOTV to hierachical index dataframe.")
noa = len(target_symbols)
if noa < 1:
    raise ValueError("no intersected symbols from specific risk and initial holding.")
logger.debug("number of asset: %s", noa)

2018-11-04 23:46:06,795 root         DEBUG    set OOTV to hierachical index dataframe.
2018-11-04 23:46:06,800 root         DEBUG    number of asset: 100


In [132]:
df_pivot_industries_asset_weights

industry   b'\xa8B7\xd6\x02W\xe9W\x02\x00\x00\x7f\x00\x00\x00\xe1'  \
symbol                   b'\xe2\x11\xeaH\x17m;\xc1"\'i\xcb+xu\x90'   
date                                                                 
2016-10-31                                           0.001828        

industry                                           \
symbol     b'\xe2\x11\xeaH891\xc1"\'i\xcb+xu\x90'   
date                                                
2016-10-31                               0.001828   

industry                                              \
symbol     b'\xe2\x11\xeaHi`\x01\xc1"\'i\xcb+xu\x90'   
date                                                   
2016-10-31                                  0.001828   

industry                                                 \
symbol     b'\xe2\x11\xeaH\xa2\xa0;\xc1"\'i\xcb+xu\x90'   
date                                                      
2016-10-31                                     0.001828   

industry                                                 \
symbol     b'\xe2\x11\xeaH\xad\xf3D\xc1"\'i\xcb+xu\x90'   
date                                                      
2016-10-31                                     0.001828   

industry                                              \
symbol     b'\xe2\x11\xeaH\xbbC@\xc1"\'i\xcb+xu\x90'   
date                                                   
2016-10-31                                  0.001828   

industry                                              \
symbol     b'\xe2\x11\xeaH\xc0E;\xc1"\'i\xcb+xu\x90'   
date                                                   
2016-10-31                                  0.001828   

industry                                                 \
symbol     b'\xe2\x11\xeaH\xe5\xb4>\xc1"\'i\xcb+xu\x90'   
date                                                      
2016-10-31                                     0.001828   

industry   b'\xa8B7\xd6\x03V\xe9W\x02\x00\x00\x7f\x00\x00\x00\xe1'  \
symbol                      b'\xe2\x11\xeaHGN<\xc1"\'i\xcb+xu\x90'   
date                                                                 
2016-10-31                                           0.001828        

industry                                                 \
symbol     b'\xe2\x11\xeaH\x8e\xa4D\xc1"\'i\xcb+xu\x90'   
date                                                      
2016-10-31                                     0.001828   

industry                             ...                            \
symbol                               ...                             
date                                 ...                             
2016-10-31                           ...                             

industry   b'\xa8B7\xd6\xf8V\xe9W\x02\x00\x00\x7f\x00\x00\x00\xe1'  \
symbol                      b'\xe2\x11\xeaH8"-\xc1"\'i\xcb+xu\x90'   
date                                                                 
2016-10-31                                           0.001828        

industry                                              \
symbol     b'\xe2\x11\xeaHW{\x17\xc1"\'i\xcb+xu\x90'   
date                                                   
2016-10-31                                  0.001828   

industry                                                 \
symbol     b'\xe2\x11\xeaH\xb7\xc5<\xc1"\'i\xcb+xu\x90'   
date                                                      
2016-10-31                                     0.001828   

industry   b'\xa8B7\xd6\xfcU\xe9W\x02\x00\x00\x7f\x00\x00\x00\xe1'  \
symbol              b'8H\x8c\x1e\xd9\x834\x02~,\x82\xce\xfeu(\x98'   
date                                                                 
2016-10-31                                           0.001828        

industry                                                   \
symbol     b'9E\x9a\xf5o\xdaw:J\xe8\xd9/\xbf\xf9\x7f\xa0'   
date                                                        
2016-10-31                                       0.001828   

industry                                           \

In [133]:
# get the ordered column list
idx_level_0_value = df_pivot_industries_asset_weights.columns.get_level_values(0)
idx_level_0_value = idx_level_0_value.drop_duplicates()
idx_level_1_value = df_pivot_industries_asset_weights.columns.get_level_values(1)
asset_return = asset_return.loc[:target_date, idx_level_1_value].fillna(0)

### 4. portfolio weight calculation

In [138]:
P = matrix(Sigma_tilde)
q = matrix(np.zeros((noa, 1)), tc='d')

A = matrix(1.0, (1, noa))
b = matrix(1.0)

# for group weight constraint
groups = df_pivot_industries_asset_weights.groupby(
    axis=1, level=0, sort=False, group_keys=False).count().\
    iloc[-1, :].values
num_group = len(groups)
num_asset = np.sum(groups)

logger.debug('number of assets in groups: %s', groups)
logger.debug('number of groups: %s', num_group)


2018-11-04 23:48:58,758 root         DEBUG    number of assets in groups: [ 8  2  1  2  2  3 11  3 14  2  6  3  1  1  5  3  3  1  6  3  4  6  3  6  1]
2018-11-04 23:48:58,766 root         DEBUG    number of groups: 25


In [144]:
# set boundary vector for h
df_asset_weight = pd.DataFrame({'lower': [0.0], 'upper': [1.0]},
                               index=idx_level_1_value)
df_group_weight = pd.DataFrame({'lower': [0.0], 'upper': [1.0]},
                               index=set(idx_level_0_value))
df_factor_exposure_bound = pd.DataFrame(index=F.T.index, columns=[['lower', 'upper']])
df_factor_exposure_bound.lower = (1.0/noa)*F.sum()*(0.999991)
df_factor_exposure_bound.upper = (1.0/noa)*F.sum()*(1.000009)

In [161]:
def logrels(rets):
    """Log of return relatives, ln(1+r), for a given DataFrame rets."""
    return np.log(rets + 1)


In [166]:
df_asset_bnd_matrix = matrix(np.concatenate(((df_asset_weight.upper,
                                              df_asset_weight.lower)), 0))
df_group_bnd_matrix = matrix(np.concatenate(((df_group_weight.upper,
                                              df_group_weight.lower)), 0))
df_factor_exposure_bnd_matrix = matrix(np.concatenate(((df_factor_exposure_bound.upper,
                                                        df_factor_exposure_bound.lower)), 0))

# Assuming AvgReturns as the expected returns if parameter is not specified
rets_mean = logrels(asset_return).mean()
avg_ret = matrix(rets_mean.values)
G = matrix(-np.transpose(np.array(avg_ret)))
h = matrix(-np.ones((1, 1))*target_return)
G_sparse_list = []
for i in range(num_group):
    for j in range(groups[i]):
        G_sparse_list.append(i)
Group_sub = spmatrix(1.0, G_sparse_list, range(num_asset))

Group_sub = matrix(sparse([Group_sub, -Group_sub]))

asset_sub = matrix(np.eye(noa))
asset_sub = matrix(sparse([asset_sub, -asset_sub]))
exp_sub = matrix(np.array(F.T))
exp_sub = matrix(sparse([exp_sub, - exp_sub]))

In [170]:
exp_sub

<60x3384 matrix, tc='d'>

#### minimum risk optimization with constraints

In [167]:
if exposure_constraint is not None:
    G0 = matrix(sparse([asset_sub, Group_sub, exp_sub]))
    h0 = matrix(sparse([df_asset_bnd_matrix, df_group_bnd_matrix,
                       df_factor_exposure_bnd_matrix]))
else:
    G0 = matrix(sparse([asset_sub, Group_sub]))
    h0 = matrix(sparse([df_asset_bnd_matrix, df_group_bnd_matrix]))

try:
    sol = solvers.qp(P, q, G0, h0, A, b)
except ValueError:
    h = matrix(-np.ones((1, 1))*100.0)
    check_constraint_issue(P, q, G, h, A, b, asset_sub, Group_sub,
                           exp_sub, df_asset_bnd_matrix,
                           df_group_bnd_matrix,
                           df_factor_exposure_bnd_matrix)
if sol['status'] == 'unknown':
    h = matrix(-np.ones((1, 1))*100.0)
    check_constraint_issue(P, q, G, h, A, b, asset_sub, Group_sub,
                           exp_sub, df_asset_bnd_matrix,
                           df_group_bnd_matrix,
                           df_factor_exposure_bnd_matrix)
df_opts_weight = pd.DataFrame(np.array(sol['x']).T,
                              columns=target_symbols,
                              index=[target_date])
logger.debug(sol['status'])
logger.debug("target return: %s", target_return)
logger.debug("all weight are bigger than 0? %s", (df_opts_weight>0).all().all())
logger.debug("all weight are smaller than 1? %s", (df_opts_weight<=1).all().all())
logger.debug("weight sum smaller than 0: %s", df_opts_weight[df_opts_weight<0].sum(1))

df_opts_weight

TypeError: incompatible dimensions of subblocks

### 5. Summary